# Catchement averaging

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os
import pcraster as pcr
import numpy as np
from netCDF4 import Dataset
import rioxarray
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re
import time
from multiprocess import Pool
import tqdm

In [2]:
# set your working directory and go there
directory = '/Users/niekcollotdescury/Desktop/Applied data science/Thesis/R_code/data/satellite_data/'
os.chdir(directory)

## Transormations cell area and ldd maps

In [19]:
os.system('cdo gridarea lwe_thickness_GRACE_data_meter.nc area/gridarea30min.nc')

0

In [7]:
# transform to tiff
os.system('gdal_translate area/gridarea30min.nc  area/gridarea30min.tif')
os.system('gdal_translate area/lddsound_30min.nc area/lddsound_30min.tif')

# transform from tiff to map (pcraster format)
os.system('gdal_translate -of PCRaster area/gridarea30min.tif  area/gridarea30min.map')
os.system('gdal_translate -of PCRaster area/lddsound_30min.tif area/lddsound_30min.map')
os.system('rm *.xml')

Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.


rm: *.xml: No such file or directory


256

In [8]:
# set your working directory and go there
directory = '/Users/niekcollotdescury/Desktop/Applied data science/Thesis/R_code/data/satellite_data/area'
os.chdir(directory)
# transform ldd map to .ldd format
os.system('pcrcalc lddsound_30min.ldd = "lddrepair(ldd(lddsound_30min.map))"')
# aguila lddsound_30min.ldd --> command in terminal for viewing the map

pcrcalc version: 4.4.0 (darwin/x86_64)


0

In [9]:
# set your working directory and go there
directory = '/Users/niekcollotdescury/Desktop/Applied data science/Thesis/R_code/data/satellite_data/'
os.chdir(directory)

In [10]:
# define cell area file and ldd file
cell_area_file = "area/gridarea30min.map"
ldd_file       = "area/lddsound_30min.ldd"

# calculate catchment area
# - set clone, the bounding box of your study area - here, we use ldd 
clone_file     = ldd_file
pcr.setclone(clone_file)
# - read cell_area and ldd files
cell_area = pcr.readmap(cell_area_file)
ldd       = pcr.readmap(ldd_file)

# - calculate catchment area
catchment_area = pcr.catchmenttotal(cell_area, ldd)
# - save catchment_area to a file - note the file output will be under the work_dir
catchment_area_file = "area/catchment_area.map"

In [21]:
pcr.report(catchment_area, catchment_area_file)

In [11]:
def check_dir_or_make(path):
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(path)


In [3]:
xds = rioxarray.open_rasterio("GHRM5C_1978-2021_float32_masked_monthly_30min.nc")
xds

<xarray.DataArray 'snow_fraction' (time: 511, y: 360, x: 720)>
[132451200 values with dtype=float32]
Coordinates:
  * time         (time) object 1978-12-15 00:00:00 ... 2021-06-15 00:00:00
  * x            (x) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * y            (y) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
    spatial_ref  int64 0
Attributes: (12/26)
    lat#axis:                      Y
    lat#long_name:                 latitude
    lat#standard_name:             latitude
    lat#units:                     degrees_north
    lon#axis:                      X
    lon#long_name:                 longitude
    ...                            ...
    missing_value:                 255
    vmax:                          1
    vmin:                          0
    _FillValue:                    255.0
    scale_factor:                  1.0
    add_offset:                    0.0

In [58]:
def create_single_timestep_maps(var, input_file, sat):
    check_dir_or_make(sat)
    check_dir_or_make(f'{sat}/timesteps')
    
    xds = rioxarray.open_rasterio(input_file)
    time = pd.DataFrame(xds[:,0,0].time.to_numpy(), columns=['date'])
    time['date'] = time['date'].astype('str')
    time[['year', 'month', 'left']]= time.date.str.split('-', expand = True)

    for i in range(len(xds[:,0,0])):
#     for i in range(10):

        timestep = time.iloc[i,:]
        name = f'{timestep.year}_{timestep.month}'
        print(i+1)
        print(name)
        
        single = f'{sat}/timesteps/{var}_{name}'

        cmd = f'cdo seltimestep,{i+1} {input_file} {single}.nc'
        os.system(cmd)

        cmd = f'gdal_translate {single}.nc {single}.tif'
        os.system(cmd)

        cmd = f"gdal_translate -of PCRaster {single}.tif {single}.map"
        os.system(cmd)
    
    os.system(f'rm {sat}/timesteps/*.xml')
    os.system(f'rm {sat}/timesteps/*.tif')
    os.system(f'rm {sat}/timesteps/*.nc')
        
input_file     = "GHRM5C_1978-2021_float32_masked_monthly_30min.nc"
create_single_timestep_maps('snow_cover', input_file,'MODIS')

input_file     = "monmean_sm_ESACCI-SOILMOISTURE-L3S-SSMV-COMBINED_1978-2019_half_degree.nc"
create_single_timestep_maps('sm', input_file,'ESA')

input_file     = "lwe_thickness_GRACE_data_meter.nc"
create_single_timestep_maps('lwe_thickness', input_file,'GRACE')


1
1978_12
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
2
1979_01
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
3
1979_02
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
4
1979_03
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
5
1979_04
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
6
1979_05
Input file size is 720, 360
0...10...20...30...40.

Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
46
1982_09
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
47
1982_10
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
48
1982_11
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
49
1982_12
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
50
1983_01
Input file size is 720, 360
0...10...20...30...40...50.

Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
90
1986_05
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
91
1986_06
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
92
1986_07
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
93
1986_08
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
94
1986_09
Input file size is 720, 360
0...10...20...30...40...50.

Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
134
1990_01
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
135
1990_02
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
136
1990_03
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
137
1990_04
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
138
1990_05
Input file size is 720, 360
0...10...20...30...40.

Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
178
1993_09
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
179
1993_10
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
180
1993_11
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
181
1993_12
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
182
1994_01
Input file size is 720, 360
0...10...20...30...40.

...

Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
459
2017_01
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
460
2017_02
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
461
2017_03
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
462
2017_04
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
463
2017_05
Input file size is 720, 360
0...10...20...30...40.

Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
9
2003_02
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
10
2003_03
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
11
2003_04
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
12
2003_05
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
13
2003_07
Input file size is 720, 360
0...10...20...30...40...50..

Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
53
2006_11
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
54
2006_12
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
55
2007_01
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
56
2007_02
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
57
2007_03
Input file size is 720, 360
0...10...20...30...40...50.

Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
97
2010_07
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
98
2010_08
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
99
2010_09
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
100
2010_10
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
101
2010_11
Input file size is 720, 360
0...10...20...30...40...5

Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
141
2015_01
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
142
2015_02
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
143
2015_03
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
144
2015_04
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 720, 360
0...10...20...30...40...50...60...70...80...90...100 - done.
145
2015_04
Input file size is 720, 360
0...10...20...30...40.

#### Possible check if the map is correct

In [57]:
# - make sure that the file has the same map attributes as the ldd file
cmd = f"mapattr -c lddsound_30min.ldd {input_file}.map"
os.system(cmd)

mapattr version: 4.4.0 (darwin/x86_64)


0

## Calculate upstream averaged values for catchment areas
Write map files to folder

In [8]:
# %%time
# def upstream_maps(sat, var):
# #     sat = u_input[0]
# #     var = u_input[1]
    
#     check_dir_or_make(f'{sat}/upstream')
    
#     input_files = glob.glob(f'{sat}/timesteps/*.map')
#     cmd = f'rm {sat}/upstream/*'
#     os.system(cmd)

#     time.sleep(5)
    
#     for i in range(len(input_files)):
        
        
        
#         name = input_files[i].split('/')[-1]
        
#         cell_value = pcr.readmap(f"{input_files[i]}")

#         # calculate upstream/catchment average values
#         upstream_average_value = pcr.catchmenttotal(cell_value * cell_area, ldd) / catchment_area
#         pcr.report(upstream_average_value, f"{sat}/upstream/upstream_{name}")
        
#         file_in = f'{sat}/upstream/upstream_{name}'
#         file_out = file_in.replace('.map', '.nc')
        
#         print(name)
        
#         cmd = f'pcr2nc -i {file_in} -o {file_out} -m {sat}/nc_metadata_1.yaml'
#         os.system(cmd)
        
# # upstream_maps('MODIS', 'snow_cover')
# # upstream_maps('ESA', 'sm')
# # upstream_maps('GRACE', 'lwe_thickness')

CPU times: user 20 µs, sys: 3 µs, total: 23 µs
Wall time: 32.9 µs


In [20]:
%%time
def upstream_maps(input_files):
    
    sat = input_files.split('/')[0]        
    name = input_files.split('/')[-1]

    
    cell_value = pcr.readmap(f"{input_files}")

    # check if cell value is present with boolean map
    cell_value_defined = pcr.defined(cell_value)
    
    # for missing values, set cell_value to zero, and cell_area (weight) to zero 
    cell_value_covered = pcr.ifthenelse(cell_value_defined, cell_value, pcr.scalar(0.0))
    cell_area_weight   = pcr.ifthenelse(cell_value_defined, cell_area, pcr.scalar(0.0))
    
    # catchment area (NOTE: this is based on cell_area_weight)
#     catchment_area = pcr.catchmenttotal(cell_value * cell_area_weight, ldd) 
    catchment_area = pcr.catchmenttotal(cell_area_weight, ldd) 
    
    # calculate upstream/catchment average values
    # calculate upstream/catchment average values (NOTE: this uses cell_value_covered and  cell_area_weight)
    upstream_average_value = pcr.catchmenttotal(cell_value_covered * cell_area_weight, ldd) / catchment_area
    pcr.report(upstream_average_value, f"{sat}/upstream/upstream_{name}")

    file_in = f'{sat}/upstream/upstream_{name}'
    file_out = file_in.replace('.map', '.nc')

    cmd = f'pcr2nc -i {file_in} -o {file_out} -m {sat}/nc_metadata_1.yaml'
    os.system(cmd)
        

CPU times: user 22 µs, sys: 32 µs, total: 54 µs
Wall time: 158 µs


In [21]:
def parralel_runs(sat, var):
    print(sat,var)
    input_files = glob.glob(f'{sat}/timesteps/*.map')


    check_dir_or_make(f'{sat}/upstream')
    cmd = f'rm {sat}/upstream/*'
    os.system(cmd)

    time.sleep(3)

    pool = Pool(processes=36) # set number of cores

    for _ in tqdm.tqdm(pool.imap_unordered(upstream_maps, input_files), total=len(input_files)):
        pass
parralel_runs('ESA', 'sm')
parralel_runs('MODIS', 'snow_cover')
parralel_runs('GRACE', 'lwe_thickness')

ESA sm


  0%|                                                   | 0/494 [00:00<?, ?it/s][09:10:32] - Defining WGS84 coordinates variables
[09:10:32] - Applying compression level 9
[09:10:32] - Creating NETCDF3_CLASSIC main variable sm: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:10:32] - Adding ESA/upstream/upstream_sm_1988_02.map - timestep None - hour 0.0
[09:10:32] - Defining WGS84 coordinates variables
[09:10:32] - Applying compression level 9
[09:10:32] - Creating NETCDF3_CLASSIC main variable sm: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:10:32] - Writing ESA/upstream/upstream_sm_1988_02.nc
[09:10:32] - Adding ESA/upstream/upstream_sm_1998_07.map - timestep None - hour 0.0
[09:10:32] - Defining WGS84 coordinates variables
[09:10:32] - Applying compression level 9
[09:10:32] - Creating NETCDF3_CLASSIC main variable sm: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:10:32] - Adding ESA/upstream/upstream_sm_1998_12.

[09:10:32] - Adding ESA/upstream/upstream_sm_1984_03.map - timestep None - hour 0.0
[09:10:32] - Writing ESA/upstream/upstream_sm_1984_03.nc
[09:10:32] - Writing ESA/upstream/upstream_sm_2012_11.nc
[09:10:32] - Defining WGS84 coordinates variables
[09:10:32] - Applying compression level 9
[09:10:32] - Creating NETCDF3_CLASSIC main variable sm: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:10:32] - Defining WGS84 coordinates variables
[09:10:32] - Applying compression level 9
[09:10:32] - Creating NETCDF3_CLASSIC main variable sm: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:10:32] - Adding ESA/upstream/upstream_sm_1984_02.map - timestep None - hour 0.0
[09:10:32] - Writing ESA/upstream/upstream_sm_1984_02.nc
[09:10:32] - Adding ESA/upstream/upstream_sm_1986_07.map - timestep None - hour 0.0
[09:10:32] - Writing ESA/upstream/upstream_sm_1986_07.nc
[09:10:32] - Defining WGS84 coordinates variables
[09:10:32] - Defining WGS84 coordinates va

[09:10:36] - Defining WGS84 coordinates variables
[09:10:36] - Applying compression level 9
[09:10:36] - Creating NETCDF3_CLASSIC main variable sm: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:10:36] - Defining WGS84 coordinates variables
[09:10:36] - Adding ESA/upstream/upstream_sm_1979_12.map - timestep None - hour 0.0
[09:10:36] - Writing ESA/upstream/upstream_sm_1979_12.nc
[09:10:36] - Applying compression level 9
[09:10:36] - Creating NETCDF3_CLASSIC main variable sm: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:10:36] - Adding ESA/upstream/upstream_sm_2002_02.map - timestep None - hour 0.0
[09:10:36] - Writing ESA/upstream/upstream_sm_2002_02.nc
  8%|███▍                                      | 41/494 [00:07<01:25,  5.30it/s][09:10:36] - Defining WGS84 coordinates variables
[09:10:36] - Applying compression level 9
[09:10:36] - Creating NETCDF3_CLASSIC main variable sm: type float32 fill value -3.4028234663852886e+38 (<class 'float

 14%|█████▉                                    | 70/494 [00:08<00:18, 22.32it/s][09:10:39] - Defining WGS84 coordinates variables
[09:10:39] - Applying compression level 9
[09:10:39] - Creating NETCDF3_CLASSIC main variable sm: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:10:39] - Adding ESA/upstream/upstream_sm_1988_11.map - timestep None - hour 0.0
[09:10:39] - Writing ESA/upstream/upstream_sm_1988_11.nc
[09:10:39] - Defining WGS84 coordinates variables
[09:10:39] - Applying compression level 9
[09:10:39] - Creating NETCDF3_CLASSIC main variable sm: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:10:39] - Adding ESA/upstream/upstream_sm_1988_05.map - timestep None - hour 0.0
[09:10:39] - Writing ESA/upstream/upstream_sm_1988_05.nc
[09:10:39] - Defining WGS84 coordinates variables
[09:10:39] - Applying compression level 9
[09:10:39] - Creating NETCDF3_CLASSIC main variable sm: type float32 fill value -3.4028234663852886e+38 (<class 'float

...

[09:12:36] - Defining WGS84 coordinates variables
[09:12:36] - Applying compression level 9
[09:12:36] - Creating NETCDF3_CLASSIC main variable lwe: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:12:36] - Adding GRACE/upstream/upstream_lwe_thickness_2005_06.map - timestep None - hour 0.0
 44%|██████████████████▎                       | 79/181 [00:11<00:17,  5.77it/s][09:12:36] - Writing GRACE/upstream/upstream_lwe_thickness_2005_06.nc
[09:12:36] - Defining WGS84 coordinates variables
[09:12:36] - Applying compression level 9
[09:12:36] - Creating NETCDF3_CLASSIC main variable lwe: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:12:37] - Adding GRACE/upstream/upstream_lwe_thickness_2009_06.map - timestep None - hour 0.0
[09:12:37] - Writing GRACE/upstream/upstream_lwe_thickness_2009_06.nc
[09:12:37] - Defining WGS84 coordinates variables
[09:12:37] - Applying compression level 9
[09:12:37] - Creating NETCDF3_CLASSIC main variable lwe: type fl

 60%|████████████████████████▍                | 108/181 [00:12<00:04, 16.69it/s][09:12:39] - Defining WGS84 coordinates variables
[09:12:39] - Applying compression level 9
[09:12:39] - Creating NETCDF3_CLASSIC main variable lwe: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:12:39] - Adding GRACE/upstream/upstream_lwe_thickness_2010_05.map - timestep None - hour 0.0
[09:12:39] - Writing GRACE/upstream/upstream_lwe_thickness_2010_05.nc
[09:12:39] - Defining WGS84 coordinates variables
[09:12:39] - Applying compression level 9
[09:12:39] - Creating NETCDF3_CLASSIC main variable lwe: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:12:39] - Adding GRACE/upstream/upstream_lwe_thickness_2010_10.map - timestep None - hour 0.0
[09:12:39] - Writing GRACE/upstream/upstream_lwe_thickness_2010_10.nc
[09:12:39] - Defining WGS84 coordinates variables
[09:12:39] - Applying compression level 9
[09:12:39] - Creating NETCDF3_CLASSIC main variable lwe: type fl

 66%|███████████████████████████▏             | 120/181 [00:15<00:07,  8.06it/s][09:12:41] - Defining WGS84 coordinates variables
[09:12:41] - Applying compression level 9
[09:12:41] - Creating NETCDF3_CLASSIC main variable lwe: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:12:41] - Defining WGS84 coordinates variables
[09:12:41] - Applying compression level 9
[09:12:41] - Creating NETCDF3_CLASSIC main variable lwe: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:12:41] - Defining WGS84 coordinates variables
[09:12:41] - Applying compression level 9
[09:12:41] - Creating NETCDF3_CLASSIC main variable lwe: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:12:41] - Adding GRACE/upstream/upstream_lwe_thickness_2006_11.map - timestep None - hour 0.0
[09:12:41] - Writing GRACE/upstream/upstream_lwe_thickness_2006_11.nc
[09:12:41] - Defining WGS84 coordinates variables
[09:12:41] - Applying compression level 9
[09:12:41] - Cre

 81%|█████████████████████████████████        | 146/181 [00:18<00:08,  4.21it/s][09:12:44] - Defining WGS84 coordinates variables
[09:12:44] - Applying compression level 9
[09:12:44] - Creating NETCDF3_CLASSIC main variable lwe: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:12:44] - Adding GRACE/upstream/upstream_lwe_thickness_2008_06.map - timestep None - hour 0.0
[09:12:44] - Writing GRACE/upstream/upstream_lwe_thickness_2008_06.nc
[09:12:44] - Defining WGS84 coordinates variables
[09:12:44] - Applying compression level 9
[09:12:44] - Creating NETCDF3_CLASSIC main variable lwe: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:12:44] - Adding GRACE/upstream/upstream_lwe_thickness_2006_06.map - timestep None - hour 0.0
[09:12:44] - Writing GRACE/upstream/upstream_lwe_thickness_2006_06.nc
[09:12:44] - Defining WGS84 coordinates variables
[09:12:44] - Applying compression level 9
[09:12:44] - Creating NETCDF3_CLASSIC main variable lwe: type fl

[09:12:45] - Writing GRACE/upstream/upstream_lwe_thickness_2008_05.nc
[09:12:45] - Adding GRACE/upstream/upstream_lwe_thickness_2004_11.map - timestep None - hour 0.0
[09:12:45] - Writing GRACE/upstream/upstream_lwe_thickness_2004_11.nc
 89%|████████████████████████████████████▍    | 161/181 [00:19<00:01, 12.76it/s][09:12:45] - Defining WGS84 coordinates variables
[09:12:45] - Applying compression level 9
[09:12:45] - Creating NETCDF3_CLASSIC main variable lwe: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:12:45] - Adding GRACE/upstream/upstream_lwe_thickness_2008_11.map - timestep None - hour 0.0
[09:12:45] - Writing GRACE/upstream/upstream_lwe_thickness_2008_11.nc
[09:12:45] - Defining WGS84 coordinates variables
[09:12:45] - Applying compression level 9
[09:12:45] - Defining WGS84 coordinates variables
[09:12:45] - Creating NETCDF3_CLASSIC main variable lwe: type float32 fill value -3.4028234663852886e+38 (<class 'float'>)
[09:12:45] - Applying compression le